In [ ]:
import torch
from transformers import BertModel,BertConfig,BertTokenizer,BertPreTrainedModel,AutoTokenizer,AutoModel
import torch.nn.functional as F
from model import AttenLSTM

bio_bert_model = AutoModel.from_pretrained('dmis-lab/biobert-large-cased-v1.1-squad')

class CustomBioBertForPipeline(BertPreTrainedModel):
    def __init__(self, config, device="gpu"):
        super().__init__(config)
        #self.bio_bert = bio_bert_model
        self.bio_bert = BertModel(config)
        self.classifier = AttenLSTM(input_size=1024, hidden_size=256, batch_first=True, dropout=0.5, bidirectional=True,
                              num_layers=2, device=device)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                inputs_embeds=None):
        outputs = self.bio_bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,
                                   position_ids=position_ids, head_mask=head_mask, inputs_embeds=inputs_embeds)
        features = F.pad(outputs.last_hidden_state, (0, 0, 0, 512 - outputs.last_hidden_state.size(1)))
        logits = self.classifier(features)
        return logits


device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
tokenizer_biobert_large = AutoTokenizer.from_pretrained('dmis-lab/biobert-large-cased-v1.1-squad')
config = BertConfig.from_pretrained("dmis-lab/biobert-large-cased-v1.1-squad", num_labels=2)
model = CustomBioBertForPipeline.from_pretrained("dmis-lab/biobert-large-cased-v1.1-squad", config=config, device=device_name).to(device)
model.eval()


def eval_one(model, input):
    item = input.replace("\n", " ").replace("  ", " ").strip()
    tokens = tokenizer_biobert_large.encode(item)
    if len(tokens) > 512:
        tokens = tokens[:512]
        print("!!!Input too long. Truncated to first 512 tokens.")
    outputs = model(torch.tensor(tokens).unsqueeze(0).to(device))
    pred = torch.max(outputs.data, 1)[1]
    (gpt_prob, hum_prob) = F.softmax(outputs.data, dim=1)[0]
    return pred[0].data, 100 * gpt_prob, 100 * hum_prob

try:
    while True:
        a = input("Please input the text to be evaluated: (0 for gpt, 1 for human)\n")
        if a == "exit":
            raise KeyboardInterrupt
        model.classifier.load_state_dict(torch.load("../Pretrained/Unified_Task1.pth", map_location=device))
        print("- Decision of GPT-Written: {}, Probability: GPT: {:.4f}%, Human: {:.4f}%.".format(*eval_one(model, a)))
        model.classifier.load_state_dict(torch.load("../Pretrained/Unified_Task2.pth", map_location=device))
        print("- Decision of GPT-Completed: {}, Probability: GPT: {:.4f}%, Human: {:.4f}%.".format(*eval_one(model, a)))
        model.classifier.load_state_dict(torch.load("../Pretrained/Unified_Task3.pth", map_location=device))
        print("- Decision of GPT-Polished: {}, Probability: GPT: {:.4f}%, Human: {:.4f}%.".format(*eval_one(model, a)))
        model.classifier.load_state_dict(torch.load("../Pretrained/Unified_Task123.pth", map_location=device))
        print("- Decision of GPT-Generated (any kind): {}, Probability: GPT: {:.4f}%, Human: {:.4f}%.\n".format(*eval_one(model, a)))
except KeyboardInterrupt:
    exit()


Some weights of BertModel were not initialized from the model checkpoint at dmis-lab/biobert-large-cased-v1.1-squad and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of CustomBioBertForPipeline were not initialized from the model checkpoint at dmis-lab/biobert-large-cased-v1.1-squad and are newly initialized: ['bert.bio_bert.encoder.layer.9.output.dense.bias', 'bert.bio_bert.encoder.layer.17.attention.self.value.bias', 'bert.bio_bert.encoder.layer.12.attention.self.value.bias', 'bert.bio_bert.encoder.layer.15.attention.self.query.bias', 'bert.bio_bert.encoder.layer.4.attention.self.value.weight', 'bert.bio_bert.encoder.layer.16.attention.self.value.weight', 'bert.bio_bert.encoder.layer.22.attention.output.LayerNorm.bias', 'bert.bio_bert.encoder.layer.9.attention.self.value.weight', 'bert.bio_bert.encoder.layer.13.output.dense.bias',

Please input the text to be evaluated: (0 for gpt, 1 for human)
  For his work discovering the basis for hemoglobin gene switching and applying those insights to develop a therapy for sickle cell disease and other blood diseases, the Institute for Regenerative Medicine at the University of Pennsylvania awarded Stuart Orkin, MD the third Elaine Redding Brinster Prize in Science or Medicine.Orkin's research advanced the understanding of how the fetal hemoglobin gene-; the main oxygen carrier protein in the human fetus-;is silenced in adults. He also developed a therapy that re-activates the fetal gene for adult hemoglobin gene defects, which cause red blood cell diseases.


- Decision of GPT-Written: 1, Probability: GPT: 27.7303%, Human: 72.2697%.
- Decision of GPT-Completed: 0, Probability: GPT: 96.6082%, Human: 3.3918%.
- Decision of GPT-Polished: 0, Probability: GPT: 78.8353%, Human: 21.1647%.
- Decision of GPT-Generated (any kind): 0, Probability: GPT: 99.9976%, Human: 0.0024%.



Please input the text to be evaluated: (0 for gpt, 1 for human)
 A new Europe-wide study investigated the prevalence of protozoans, bacteria and viruses potentially pathogenic to humans and domestic animals in birds and bats in varying climatic conditions. The prevalence of many of these pathogens was associated with temperature or rainfall.The new study compiled information on the occurrence of over 75 pathogenic microbes across Europe from almost 400 bird- and 40 bat species. Combining data on occurrence with climatic factors revealed that the occurrence of most pathogens was associated with temperature or rainfall.


- Decision of GPT-Written: 1, Probability: GPT: 29.8110%, Human: 70.1889%.
- Decision of GPT-Completed: 1, Probability: GPT: 26.9345%, Human: 73.0655%.
- Decision of GPT-Polished: 0, Probability: GPT: 81.9400%, Human: 18.0600%.
- Decision of GPT-Generated (any kind): 0, Probability: GPT: 99.9794%, Human: 0.0206%.



...

- Decision of GPT-Written: 1, Probability: GPT: 47.5171%, Human: 52.4829%.
- Decision of GPT-Completed: 0, Probability: GPT: 81.5819%, Human: 18.4181%.
- Decision of GPT-Polished: 1, Probability: GPT: 18.0399%, Human: 81.9601%.
- Decision of GPT-Generated (any kind): 0, Probability: GPT: 99.9507%, Human: 0.0493%.



Please input the text to be evaluated: (0 for gpt, 1 for human)
 EBADNR leverages the power of model interpretability and explanation techniques to identify adversarial samples by scrutinizing the underlying decision rationale of machine learning models. By extracting feature importance and decision rationales from interpretability methods like LIME or SHAP, EBADNR creates a fingerprint of normal data behavior. When presented with input data, it compares the model's decision rationale with the expected explanation. Deviations from the expected rationale are indicative of adversarial perturbations.


- Decision of GPT-Written: 1, Probability: GPT: 47.5283%, Human: 52.4717%.
- Decision of GPT-Completed: 0, Probability: GPT: 95.6903%, Human: 4.3097%.
- Decision of GPT-Polished: 0, Probability: GPT: 79.9368%, Human: 20.0632%.
- Decision of GPT-Generated (any kind): 0, Probability: GPT: 99.8963%, Human: 0.1037%.



Please input the text to be evaluated: (0 for gpt, 1 for human)
 Adversarial attacks pose a substantial threat to the reliability and security of machine learning models, particularly in critical applications such as image classification and natural language processing. This research paper introduces a novel approach, Explanation-Based Adversarial Detection with Noise Reduction (EBAD-NR), aimed at enhancing the robustness of machine learning models against adversarial attacks.


- Decision of GPT-Written: 0, Probability: GPT: 73.7338%, Human: 26.2662%.
- Decision of GPT-Completed: 0, Probability: GPT: 96.9953%, Human: 3.0047%.
- Decision of GPT-Polished: 0, Probability: GPT: 81.8635%, Human: 18.1365%.
- Decision of GPT-Generated (any kind): 0, Probability: GPT: 99.9628%, Human: 0.0372%.

